## HW1. Проектирование API
Автор: *Кузнецов Кирилл Игоревич*, группа, дата

Это задание выполняется в рамках модуля 1 «Проектирование API». Вы закрепите навыки разработки API, используя подход сode-first, затем будете придерживаться подхода API-first

> Чтобы получить максимальный балл, убедитесь, что ваш ноутбук запускается с нуля, структура понятна, а в выводах вы объясняете свои решения.  

## Подготовка окружения


1. Работа производилась на локальной машине

2. В процессе реализации была создана виртуальная среда
`conda create --name devops_env  python=3.11 `

In [ ]:
# Для установки в zsh и обхода спецссимволов немного модернизируем команду
# !pip install 'fastapi[all]' 'uvicorn[standard]'

Теперь, когда у нас установлены необходимые библиотеки, мы можем приступить к созданию нашего первого приложения FastAPI.

### Задание 1

Вам необходимо самостоятельно создать веб-сервер на основе FastAPI.

Задача: создать файл `main.py`, который будет содержать наш код API с четырьмя методами HTTP.



### Задание 2

Сервер должен отвечать валидным JSON на эндпоинте /json_data.

Задача: создать новый эндпоинт /json_data и подключить компонент JSONResponse на этом эндпоинте. Содержание JSONa не важно, главное, чтобы он был валидным.


### Задание 3

Обработка ошибок с использованием простого HTTPException.

Задачи:
1. Изучите стандартные коды ошибок.
2. Выберите любое случайное число в диапазоне 400—526.

> Чтобы возвращать ошибки с соответствующими HTTP-статусами вам нужно:
> 1. подключить класс `HTTPException` и просто выдать любой [код ошибки](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP) (от 400 до 526);
> 2. создать маршрут /error, который будет генерировать ошибку с этим кодом в ответ на любой запрос.

В комментариях напишите, как вы понимаете, что означает выбранный вами код ошибки.

In [ ]:
import random;print("используйте и опишите код ошибки", random.randint(400,526))

511


### Задание 4

Создание автодокументации для API: FastAPI автоматически генерирует документацию API в формате OpenAPI и предоставляет интерфейс Swagger UI для ее просмотра

*Поскольку код коллаба выполняется внутри виртуальной машины, у которой нет внешнего IP-адреса, вам потребуется создать тоннель, чтобы получить внешний IP-адрес.*

*Зарегистрируйтесь в личном кабинете https://xtunnel.ru/ и скопируйте бесплатную лицензию (секретный ключ API)*

Если возникают сложности, используйте локальную версию коллаба.
```bash
pip install notebook
jupyter notebook
```


Задача. Чтобы проверить документацию, выполните следующие шаги:
1. Запустите сервер FastAPI с помощью команды ниже.
2. Откройте браузер и перейдите по адресу `http://127.0.0.1:8000/docs` для просмотра документации в Swagger UI.
3. Для просмотра документации в формате OpenAPI перейдите по адресу `http://127.0.0.1:8000/openapi.json`

In [ ]:
"""

вставьте сюда полученый JSON, который вы скопировали по адресу http://127.0.0.1:8000/openapi.json

"""

### Задание 5

Автодокументаци API хороша для маленьких проектов. Сейчас вам нужно полученый в задании 4 JSON вставить в редактор [Swagger](https://editor.swagger.io/), добавить проектируемый маршрут с ответом в формате XML и сохранить описание в YAML.

Задачи:
1. Откройте редактор Swagger, вставьте JSON (ответьте ОК на запрос Would you like to convert your JSON into YAML?)
2. Добавьте 1 эндпоинт.
3. Скопируйте получившееся описание в YAML и вставьте в ячейку ниже.

In [ ]:
"""

вставьте сюда полученый YAML, который вы скопировали из SWagger

"""

## Итоговое оформление


1. Подготовьте ноутбук в логичной структуре: написание кода → работа с JSON → обработка ошибок → API в YAML → итоги.  
2. В ячейках Markdown сформулируйте 5–8 предложений с выводами, когда стоит применять подход Code-first и почему стоит придерживаться подхода API-first.  

